In [1]:
from manim import *
import jupyter_capture_output
from numpy import linalg as npl

video_scene = " -v WARNING --disable_caching belastung_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s belastung_Scene"

Manim Community v0.18.0.post0

Jupyter Capture Output v0.0.11


In [2]:
stress_data = np.array([
        [1, 35.66990291, 1.453013591, 1],

        [2, 38.52083333, 2.159834341, 2],

        [3, 35.61016949, 1.745666246, 3],

        [4, 45.515625, 1.31475876, 4],

        [5, 50.09876543, 1.88296497, 5],

        [6, 58.44444444, 2.285782927, 6],

        [7, 54.56790123, 1.694484359, 7],

        [8, 56.61971831, 1.888583321, 8],

        [9, 56.67948718, 1.975330167, 9],

        [10, 56.18181818, 1.926373102, 10],

        [11, 59.08888889, 2.577059287, 11],

        [12, 56.27777778, 2.570325322, 12],

        [13, 55.14814815, 4.986221386, 13],

        [14, 57.34883721, 1.821273246, 14],

        [15, 60.54237288, 1.971053238, 15],

        [16, 53.92405063, 1.773783271, 16],

        [17, 55.53731343, 1.959486586, 17],

        [18, 62.66666667, 2.584519019, 18],

        [19, 63.60869565, 2.789547919, 19],

        [20, 40.64705882, 5.487245356, 20],

        [21, 26.6, 4.787682609, 21],

        [22, 32.5, 5.306796389, 22],

        [23, 20.16666667, 6.477739661, 23],
])

In [7]:
class BelastungTrajectorie(Mobject):
    def __init__(self, center = np.array([0, 0, 0]), x_range = [0, 23.9, 1], y_range = [0, 79, 20], x_length = 10, y_length = 5, **kwargs):
        super().__init__(**kwargs)
        self.uniblau = ManimColor.from_rgb((21,50,104))

        x_coord = [i for i in range(1, 24)]
        y_coord = [20*i for i in range(4)]
        y_label_coord = [20*i for i in range(4)]

        x_dict = dict(zip(x_coord, x_coord))
        y_dict = dict(zip(y_coord, y_label_coord))

        self.ax = Axes(x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length,
            axis_config = {"stroke_opacity": 1, "stroke_color": BLACK, "font_size": 24, "tip_width": 0.15, "tip_height": 0.15},# "font": "Times New Roman"}, 
            # y_axis_config = {"stroke_opacity": 1, "stroke_color": BLACK, "font_size": 24}, 
            x_axis_config = {"include_ticks": True, "tick_size": 0.001, "numbers_with_elongated_ticks": [0, 3, 18, 20, 23], "longer_tick_multiple": 100},
        ).move_to(center).set_color(BLACK)

        self.ax.y_axis.add_numbers(y_dict).set_color(BLACK)

        x_label = Text("Course of the semester", color = BLACK, font_size = 18, font = "Arial").next_to(self.ax.x_axis.get_center(), 2.5*DOWN)
        y_label = Text("Perceived stress from 0 (min) to 100 (max)", color = BLACK, font_size = 18, font = "Arial").next_to(self.ax.y_axis.get_center(), RIGHT+UP)
        self.add(self.ax)#, x_label)#, y_label)


    def get_dot(self, time, stress, std = False):
        time = time-0.5
        stress_dot = Dot(radius = 0.05, color = self.uniblau).move_to(self.ax.c2p(time, stress))
        stress_group = VGroup()
        stress_group.add(stress_dot)
        if (std):
            std_vert_line = Line(start = self.ax.c2p(time, stress-std), end = self.ax.c2p(time, stress+std), stroke_color = self.uniblau, stroke_width = 1)
            std_top_line = Line(start = self.ax.c2p(time-0.1, stress+std), end = self.ax.c2p(time+0.1, stress+std), stroke_color = self.uniblau, stroke_width = 1)
            std_bottom_line = Line(start = self.ax.c2p(time-0.1, stress-std), end = self.ax.c2p(time+0.1, stress-std), stroke_color = self.uniblau, stroke_width = 1)
            stress_group.add(std_vert_line, std_top_line, std_bottom_line)
        return stress_group
    

    def get_line(self, time, stress):
        time_start = time[0]-0.5
        time_end = time[1]-0.5
        line = Line(start = self.ax.c2p(time_start, stress[0]), end = self.ax.c2p(time_end, stress[1]), stroke_color = self.uniblau, stroke_width = 2)
        return line
    

    def get_period(self, time, polygon_opacity):
        top = 79
        period_rect = Polygon(
            self.ax.c2p(time[0], 0),
            self.ax.c2p(time[0], top),
            self.ax.c2p(time[1], top),
            self.ax.c2p(time[1], 0),
            color = GRAY, fill_color = GRAY, stroke_opacity = 0, fill_opacity = polygon_opacity
        )
        return period_rect
    

    def get_descriptor_text(self, time, descriptor, offset = 0):
        descriptor_text = Text(descriptor, color = BLACK, font_size = 72).move_to(self.ax.c2p((time[1]+time[0])/2, -4+offset)).scale(0.25)
        return descriptor_text

In [9]:
%%capture_video --path "animations/belastung/belastung_periods_sans.mp4"
%%manim -qh --fps 60 $video_scene


class belastung_Scene(Scene):
    def construct(self):
        self.camera.background_color = WHITE

        headline = Text("Perceived stress from 0 (min) to 100 (max)", font = "Arial", font_size = 48, color = BLACK).align_on_border(UP, buff = 0.75).scale(0.5)
        self.add(headline)

        ax_belastung = BelastungTrajectorie(center = np.array([0, 0, 0]))
        self.add(ax_belastung)

        text_pre = ax_belastung.get_descriptor_text([0, 3], "Precourse")
        text_lw = ax_belastung.get_descriptor_text([3, 18], "Lecture weeks")
        text_ew = ax_belastung.get_descriptor_text([18, 20], "Exams")
        text_free = ax_belastung.get_descriptor_text([20, 23], "Free")
        self.add(text_pre, text_lw, text_ew, text_free)


        # play parameters
        dot_fade = 0.15

        # play stress data
        self.wait(0.5)
        stress_dot = ax_belastung.get_dot(stress_data[0][0], stress_data[0][1], stress_data[0][2])
        period_rect = ax_belastung.get_period([0, 3], 0.125)
        self.play(FadeIn(stress_dot), FadeIn(period_rect), run_time = dot_fade)
        for x in range(1, len(stress_data)):
            stress_dot = ax_belastung.get_dot(stress_data[x][0], stress_data[x][1], stress_data[x][2])
            stress_line = ax_belastung.get_line([stress_data[x-1][0], stress_data[x][0]], [stress_data[x-1][1], stress_data[x][1]])
            # if (x == 1):
            #     self.play(FadeIn(stress_dot), FadeIn(stress_line), FadeIn(period_rect), run_time = dot_fade)
            if (x == 3):
                period_rect = ax_belastung.get_period([3, 18], 0.25)
                self.play(FadeIn(stress_dot), FadeIn(stress_line), FadeIn(period_rect), run_time = dot_fade)
            elif (x == 18):
                period_rect = ax_belastung.get_period([18, 20], 0.125)
                self.play(FadeIn(stress_dot), FadeIn(stress_line), FadeIn(period_rect), run_time = dot_fade)       
            elif (x == 21):
                period_rect = ax_belastung.get_period([20, 23], 0.25)
                self.play(FadeIn(stress_dot), FadeIn(stress_line), FadeIn(period_rect), run_time = dot_fade)
            else:
                self.play(FadeIn(stress_dot), FadeIn(stress_line), run_time = dot_fade)

        self.wait(1)

Output saved by overwring previous file at animations/belastung/belastung_periods_sans.mp4.
